In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import glob
import calendar
import datetime

path = r'.\Data' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

c:\Users\tijnv\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\tijnv\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\tijnv\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
frame.shape

(5355214, 25)

## Data preprocessing
At  the moment we have decided to filter the busiest airline and its top-3 most used departure and arrival airports. 

### Busiest airline

In [3]:
busiest_airline_id = frame['OP_CARRIER_AIRLINE_ID'].value_counts().head(1).index[0]
busiest_airline_id

19393

In [4]:
airline_ids = pd.read_csv('airport_airline_data/L_AIRLINE_ID.csv')
airline_ids[airline_ids['Code']==busiest_airline_id]

,Code,Description
362,19393,Southwest Airlines Co.: WN


### Top-3 departure and arrival airports

In [5]:
airports_filter = []
three_departure = frame[frame['OP_CARRIER_AIRLINE_ID']==busiest_airline_id]['ORIGIN_AIRPORT_ID'].value_counts().head(3).index.to_list()
three_arrival = frame[frame['OP_CARRIER_AIRLINE_ID']==busiest_airline_id]['DEST_AIRPORT_ID'].value_counts().head(3).index.to_list()
airports_filter.extend(x for x in three_departure if x not in airports_filter)
airports_filter.extend(x for x in three_arrival if x not in airports_filter)
airports_filter

[13232, 11292, 12889]

In [6]:
airport_ids = pd.read_csv('airport_airline_data/L_AIRPORT_ID.csv')
airport_ids[airport_ids.Code.isin(airports_filter)]

,Code,Description
1245,11292,"Denver, CO: Denver International"
2762,12889,"Las Vegas, NV: McCarran International"
3098,13232,"Chicago, IL: Chicago Midway International"


### Filter busiest airline with its top-3 used departure and arrival airports

In [7]:
busiest_airline_df = frame[frame['OP_CARRIER_AIRLINE_ID']==busiest_airline_id]
filtered_df = busiest_airline_df[(busiest_airline_df.DEST_AIRPORT_ID.isin(airports_filter)) | (busiest_airline_df.ORIGIN_AIRPORT_ID.isin(airports_filter))]
filtered_df

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
116754,2018,4,2,1,19393,13232,13871,2000,2026.0,26.0,...,90.0,87.0,1.0,423.0,2,0.0,0.0,0.0,0.0,23.0
116755,2018,4,2,1,19393,13232,13871,825,818.0,-7.0,...,90.0,88.0,1.0,423.0,2,NaN,NaN,NaN,NaN,NaN
116756,2018,4,2,1,19393,13232,13871,1015,1021.0,6.0,...,85.0,85.0,1.0,423.0,2,NaN,NaN,NaN,NaN,NaN
116757,2018,4,2,1,19393,13232,13891,2005,2005.0,0.0,...,260.0,252.0,1.0,1706.0,7,NaN,NaN,NaN,NaN,NaN
116758,2018,4,2,1,19393,13232,13931,2305,2312.0,7.0,...,110.0,102.0,1.0,704.0,3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5355194,2022,7,13,3,19393,11638,11292,1455,1458.0,3.0,...,135.0,133.0,1.0,844.0,4,NaN,NaN,NaN,NaN,NaN
5355195,2022,7,13,3,19393,11638,12889,1550,1544.0,-6.0,...,70.0,68.0,1.0,259.0,2,NaN,NaN,NaN,NaN,NaN
5355196,2022,7,13,3,19393,11638,12889,1115,1226.0,71.0,...,70.0,77.0,1.0,259.0,2,0.0,0.0,7.0,0.0,71.0
5355197,2022,7,13,3,19393,11638,12889,555,549.0,-6.0,...,65.0,65.0,1.0,259.0,2,NaN,NaN,NaN,NaN,NaN


### Remove cancelled and diverted flights

In [8]:
filtered_df = filtered_df[(filtered_df['CANCELLED']!=1)|(filtered_df['DIVERTED']!=1)]

### Transform month to string (categorical variable)

In [9]:
filtered_df["MONTH_STRING"] = filtered_df["MONTH"].apply(lambda x: calendar.month_name[int(x)])

### Make year a dummy variable

In [10]:
dummies_year = pd.get_dummies(filtered_df['YEAR'])
filtered_df = pd.concat([filtered_df, dummies_year], axis=1)

### Convert CRS_DEP_TIME and CRS_ARR_TIME

In [11]:
def convert_time(time_int):
    time_str = str(time_int)
    if len(time_str) == 1:
        time_str = "00:0" + time_str
    elif len(time_str) == 2:
        time_str = '00:' + time_str
    elif len(time_str) == 3:
        time_str = '0' + time_str[0] + ':' + time_str[1:]
    elif len(time_str) == 4:
        time_str = time_str[:2] + ':' + time_str[2:]
    else:
        print(time_str)
    time = datetime.datetime.strptime(time_str, "%H:%M")
    time_delta_start_of_day = datetime.timedelta(hours=time.hour, minutes=time.minute)
    time_delta_minutes = time_delta_start_of_day.total_seconds()/60
    return time_delta_minutes

In [12]:
filtered_df['CRS_DEP_TIME_DELTA'] = filtered_df['CRS_DEP_TIME'].apply(lambda x: convert_time(x))

In [13]:
filtered_df['CRS_ARR_TIME_DELTA'] = filtered_df['CRS_ARR_TIME'].apply(lambda x: convert_time(x))

## Exploration

In [14]:
filtered_df[(filtered_df['CANCELLED']==1)|(filtered_df['DIVERTED']==1)]

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,MONTH_STRING,2018,2020,2022,CRS_DEP_TIME_DELTA,CRS_ARR_TIME_DELTA
117696,2018,4,19,4,19393,14771,12889,2145,NaN,NaN,...,NaN,NaN,NaN,NaN,April,1,0,0,1305.0,1395.0
119597,2018,4,23,1,19393,14107,12889,2025,NaN,NaN,...,NaN,NaN,NaN,NaN,April,1,0,0,1225.0,1290.0
119602,2018,4,23,1,19393,14107,12889,1640,NaN,NaN,...,NaN,NaN,NaN,NaN,April,1,0,0,1000.0,1060.0
119624,2018,4,23,1,19393,14107,13232,730,NaN,NaN,...,NaN,NaN,NaN,NaN,April,1,0,0,450.0,765.0
119871,2018,4,23,1,19393,14679,11292,1600,1708.0,68.0,...,NaN,NaN,NaN,NaN,April,1,0,0,960.0,1155.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5352346,2022,7,6,3,19393,13232,15016,905,NaN,NaN,...,NaN,NaN,NaN,NaN,July,0,0,1,545.0,615.0
5352511,2022,7,6,3,19393,13796,11292,2140,NaN,NaN,...,NaN,NaN,NaN,NaN,July,0,0,1,1300.0,60.0
5353325,2022,7,6,3,19393,14831,11292,2130,NaN,NaN,...,NaN,NaN,NaN,NaN,July,0,0,1,1290.0,55.0
5353435,2022,7,6,3,19393,14869,11292,1655,NaN,NaN,...,NaN,NaN,NaN,NaN,July,0,0,1,1015.0,1100.0


In [15]:
# Number of flights that had delay due to carrier 
filtered_df[(filtered_df['CARRIER_DELAY']>0)].shape

(58253, 31)

In [16]:
# Number of flights that had delay due to NAS 
filtered_df[filtered_df['NAS_DELAY']>0].shape

(32468, 31)

In [17]:
# Number of flights that had delay due to late aircraft 
filtered_df[(filtered_df['LATE_AIRCRAFT_DELAY']>0)].shape

(55625, 31)

In [18]:
# Number of flights that had delay due to carrier and NAS 
filtered_df[(filtered_df['CARRIER_DELAY']>0)&(filtered_df['NAS_DELAY']>0)].shape

(18328, 31)

In [19]:
# Number of flights that had delay due to carrier and late aircraft 
filtered_df[(filtered_df['CARRIER_DELAY']>0)&(filtered_df['LATE_AIRCRAFT_DELAY']>0)].shape

(39512, 31)

In [20]:
# Number of flights that had delay due to NAS and late aircraft
filtered_df[(filtered_df['NAS_DELAY']>0)&(filtered_df['LATE_AIRCRAFT_DELAY']>0)].shape

(17208, 31)

In [21]:
# Number of flights that had delay due to carrier, NAS and late aircraft
filtered_df[(filtered_df['CARRIER_DELAY']>0)&(filtered_df['LATE_AIRCRAFT_DELAY']>0)&(filtered_df['NAS_DELAY']>0)].shape

(11056, 31)

In [22]:
# Check type of delays compare to departure and arrival delay
filtered_df[filtered_df['CARRIER_DELAY']>=0][['CRS_DEP_TIME','DEP_TIME','DEP_DELAY','CRS_ARR_TIME','ARR_TIME','ARR_DELAY','CARRIER_DELAY','NAS_DELAY','LATE_AIRCRAFT_DELAY']]

,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CARRIER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY
116754,2000,2026.0,26.0,2130,2153.0,23.0,0.0,0.0,23.0
116764,1035,1038.0,3.0,1325,1358.0,33.0,0.0,33.0,0.0
116772,945,1011.0,26.0,1130,1148.0,18.0,5.0,0.0,13.0
116777,1005,1027.0,22.0,1220,1240.0,20.0,0.0,0.0,20.0
116778,720,801.0,41.0,940,1019.0,39.0,12.0,0.0,27.0
...,...,...,...,...,...,...,...,...,...
5355138,1540,1606.0,26.0,1810,1838.0,28.0,10.0,2.0,16.0
5355139,1115,1143.0,28.0,1355,1415.0,20.0,19.0,0.0,1.0
5355140,1950,2013.0,23.0,2225,2246.0,21.0,0.0,0.0,21.0
5355181,1705,1746.0,41.0,1750,1830.0,40.0,1.0,0.0,39.0


In [23]:
#https://bestcarsanswers.com/library/article/read/32856-what-is-crs-elapsed-time-of-flight
filtered_df['DELAYED'] = (filtered_df['DEP_DELAY']>=15)|(filtered_df['ARR_DELAY']>=15)
filtered_df.head(2)

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,MONTH_STRING,2018,2020,2022,CRS_DEP_TIME_DELTA,CRS_ARR_TIME_DELTA,DELAYED
116754,2018,4,2,1,19393,13232,13871,2000,2026.0,26.0,...,0.0,0.0,23.0,April,1,0,0,1200.0,1290.0,True
116755,2018,4,2,1,19393,13232,13871,825,818.0,-7.0,...,NaN,NaN,NaN,April,1,0,0,505.0,595.0,False


In [24]:
def categories_cat_delay(series: pd.Series) -> list:
    lst = []
    for value in series:
        if value < 1:
            if value != 0:
                print(value)
            lst.append(0)
        elif value > 0: 
            lst.append('Greater than 0')
        elif pd.isna(value):
            lst.append('nan')
        else:
            print(value)
    return lst

In [25]:
filtered_df['CARRIER_DELAY_BOOL'] = categories_cat_delay(filtered_df['CARRIER_DELAY'])

In [26]:
#This output shows whether if a flight is delayed (has a departure or arrival delay >= 15), it has a value within a categorical delay column
#If there is a value in one of the categorical delay columns all categorical delay columns have a value <- True
#Therefore this output only shows it for carrier_delay column too make it more easy to read
filtered_df[['DELAYED','CARRIER_DELAY_BOOL']].value_counts()

DELAYED  CARRIER_DELAY_BOOL
False    nan                   292293
True     Greater than 0         58253
         0                      24619
         nan                    24562
dtype: int64

### Extra preprocessing based on exploration

In [27]:
# ^ There are 24562 rows that are delayed but do not have values for the reasoning behind this (values in the categorical delay columns)
# Therefore delete these rows! Do not have information about the categorical delay columns eventhough these flights are delayed.
filtered_df2 = filtered_df[~((filtered_df['DELAYED']==True) & (filtered_df['CARRIER_DELAY_BOOL'] == 'nan'))]

### Further exploration

In [28]:
expl_df = filtered_df2[['CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY','DELAYED']].copy()

In [29]:
expl_df['CARRIER_DELAY_BOOL'] = categories_cat_delay(expl_df['CARRIER_DELAY'])
expl_df['WEATHER_DELAY_BOOL'] = categories_cat_delay(expl_df['WEATHER_DELAY'])
expl_df['NAS_DELAY_BOOL'] = categories_cat_delay(expl_df['NAS_DELAY'])
expl_df['SECURITY_DELAY_BOOL'] = categories_cat_delay(expl_df['SECURITY_DELAY'])
expl_df['LATE_AIRCRAFT_DELAY_BOOL'] = categories_cat_delay(expl_df['LATE_AIRCRAFT_DELAY'])

In [30]:
expl_df[['DELAYED','CARRIER_DELAY_BOOL','WEATHER_DELAY_BOOL','NAS_DELAY_BOOL','SECURITY_DELAY_BOOL','LATE_AIRCRAFT_DELAY_BOOL']].value_counts()

DELAYED  CARRIER_DELAY_BOOL  WEATHER_DELAY_BOOL  NAS_DELAY_BOOL  SECURITY_DELAY_BOOL  LATE_AIRCRAFT_DELAY_BOOL
False    nan                 nan                 nan             nan                  nan                         292293
True     Greater than 0      0                   0               0                    Greater than 0               28456
                                                                                      0                            11469
                                                 Greater than 0  0                    Greater than 0               11056
         0                   0                   0               0                    Greater than 0                9289
                                                 Greater than 0  0                    0                             7506
         Greater than 0      0                   Greater than 0  0                    0                             7272
         0                   0            

### Fill nan with 0 

In [31]:
filtered_df2 = filtered_df2.fillna(0)

In [32]:
filtered_df2.to_csv('DSSD_input/total_carrier.csv')

In [33]:
filtered_df2.shape

(375165, 33)

# Carrier
Carrier df including flights with and without delay, X are variables that are known beforehand about the flight and Y is the delay in minutes due to the carrier

In [34]:
X_columns_total = [2018,2020,2022,'MONTH_STRING','DAY_OF_MONTH','DAY_OF_WEEK','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','CRS_DEP_TIME_DELTA','CRS_ARR_TIME_DELTA','CRS_ELAPSED_TIME','DISTANCE']
X_columns_p_year = ['MONTH_STRING','DAY_OF_MONTH','DAY_OF_WEEK','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','CRS_DEP_TIME_DELTA','CRS_ARR_TIME_DELTA','CRS_ELAPSED_TIME','DISTANCE']
#,'DISTANCE_GROUP', 'YEAR', 'CRS_DEP_TIME', 'CRS_ARR_TIME','OP_CARRIER_AIRLINE_ID'

## Carrier total

In [35]:
X_carrier = filtered_df2[X_columns_total].copy()

In [36]:
X_carrier.shape

(375165, 12)

In [37]:
y_carrier = filtered_df2[['CARRIER_DELAY']]

## Carrier 2018

In [38]:
X_carrier_2018 = filtered_df2[filtered_df2['YEAR'] == 2018][X_columns_p_year].copy()

In [39]:
X_carrier_2018.shape

(139736, 9)

In [40]:
y_carrier_2018 = filtered_df2[filtered_df2['YEAR'] == 2018][['CARRIER_DELAY']].copy()

## Carrier 2020

In [41]:
X_carrier_2020 = filtered_df2[filtered_df2['YEAR'] == 2020][X_columns_p_year].copy()

In [42]:
X_carrier_2020.shape

(100217, 9)

In [43]:
y_carrier_2022 = filtered_df2[filtered_df2['YEAR'] == 2020][['CARRIER_DELAY']].copy()

## Carrier 2022

In [44]:
X_carrier_2022 = filtered_df2[filtered_df2['YEAR'] == 2022][X_columns_p_year].copy()

In [45]:
X_carrier_2022.shape

(135212, 9)

In [46]:
y_carrier_2022 = filtered_df2[filtered_df2['YEAR'] == 2022][['CARRIER_DELAY']].copy()